In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

stl_path = "../processed_data/stl_energy_production_with_engineered_features.csv"
hp_path = "../processed_data/hp_energy_production_with_engineered_features.csv"

stl_df = pd.read_csv(stl_path)
hp_df = pd.read_csv(hp_path)

feature_cols = [
    "Water_Flow_m3_s", "avgtempC", "totalprecipMM", "humidity", "pressureMB",
    "WaterFlow_Diff_1d", "WaterFlow_Diff_7d", "WaterFlow_3day_avg", "WaterFlow_7day_avg",
    "Temp_Deviation", "WaterFlow_Humidity", "month_sin", "month_cos",
    "Normalized_Efficiency", "Prev_Day_Efficiency", "Prev_Week_Efficiency"
]
target_col = "Efficiency"

def prepare_data(df):
    X = df[feature_cols].values
    y = df[target_col].values.reshape(-1, 1)

    scaler_X = StandardScaler()
    scaler_y = StandardScaler()
    X = scaler_X.fit_transform(X)
    y = scaler_y.fit_transform(y)

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    return (
        torch.tensor(X_train, dtype=torch.float32), torch.tensor(X_test, dtype=torch.float32),
        torch.tensor(y_train, dtype=torch.float32), torch.tensor(y_test, dtype=torch.float32),
        scaler_y
    )

stl_X_train, stl_X_test, stl_y_train, stl_y_test, stl_scaler_y = prepare_data(stl_df)
hp_X_train, hp_X_test, hp_y_train, hp_y_test, hp_scaler_y = prepare_data(hp_df)

batch_size = 64
stl_train_loader = DataLoader(TensorDataset(stl_X_train, stl_y_train), batch_size=batch_size, shuffle=True)
stl_test_loader = DataLoader(TensorDataset(stl_X_test, stl_y_test), batch_size=batch_size, shuffle=False)

hp_train_loader = DataLoader(TensorDataset(hp_X_train, hp_y_train), batch_size=batch_size, shuffle=True)
hp_test_loader = DataLoader(TensorDataset(hp_X_test, hp_y_test), batch_size=batch_size, shuffle=False)

In [2]:
class HydropowerEfficiencyNN(nn.Module):
    def __init__(self, input_dim):
        super(HydropowerEfficiencyNN, self).__init__()

        self.model = nn.Sequential(
            nn.Linear(input_dim, 128),
            nn.BatchNorm1d(128),
            nn.GELU(),  
            nn.Dropout(0.2),

            nn.Linear(128, 64),
            nn.BatchNorm1d(64),
            nn.SiLU(),  
            nn.Dropout(0.2),

            nn.Linear(64, 32),
            nn.BatchNorm1d(32),
            nn.ReLU(),
            nn.Dropout(0.1),

            nn.Linear(32, 16),
            nn.BatchNorm1d(16),
            nn.ReLU(),
            nn.Dropout(0.1),

            nn.Linear(16, 1)  
        )

    def forward(self, x):
        if x.dim() == 3:
            x = x[:, -1, :]
        out = self.model(x)
        return out


input_dim = len(feature_cols)
device = torch.device("cpu")  
model = HydropowerEfficiencyNN(input_dim=len(feature_cols)).to(device)

In [3]:
criterion = nn.MSELoss()
optimizer = optim.AdamW(model.parameters(), lr=0.0005, weight_decay=1e-5)
scheduler = optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=5, T_mult=2)  

In [4]:
def train_model(model, train_loader, test_loader, num_epochs=50, patience=7):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    criterion = nn.MSELoss()
    optimizer = optim.AdamW(model.parameters(), lr=0.0005, weight_decay=1e-4)  # Increased weight decay
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=3, verbose=True)  

    train_losses, test_losses = [], []
    best_test_loss = float("inf")
    patience_counter = 0

    for epoch in range(num_epochs):
        model.train()
        epoch_loss = 0
        for X_batch, y_batch in train_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)

            optimizer.zero_grad()
            y_pred = model(X_batch)
            loss = criterion(y_pred, y_batch)
            loss.backward()
            optimizer.step()

            epoch_loss += loss.item()
        
        train_losses.append(epoch_loss / max(1, len(train_loader)))

        model.eval()
        test_loss = 0
        with torch.no_grad():
            for X_batch, y_batch in test_loader:
                X_batch, y_batch = X_batch.to(device), y_batch.to(device)
                y_pred = model(X_batch)
                test_loss += criterion(y_pred, y_batch).item()
        
        test_loss /= max(1, len(test_loader))
        test_losses.append(test_loss)

        scheduler.step(test_loss)

        if test_loss < best_test_loss:
            best_test_loss = test_loss
            patience_counter = 0 
        else:
            patience_counter += 1
            if patience_counter >= patience:
                print(f"Early stopping triggered at epoch {epoch+1}")
                break

        if (epoch + 1) % 5 == 0:
            print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {train_losses[-1]:.4f}, Test Loss: {test_losses[-1]:.4f}")

    return train_losses, test_losses
train_losses_stl, test_losses_stl = train_model(model, stl_train_loader, stl_test_loader, num_epochs=50)
train_losses_hp, test_losses_hp = train_model(model, hp_train_loader, hp_test_loader, num_epochs=50)

/Users/elvinanosrati/Capstone/energy-production-analysis/myenv-py311/lib/python3.11/site-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Epoch 5/50, Train Loss: 0.0197, Test Loss: 3.9736
Epoch 10/50, Train Loss: 0.0129, Test Loss: 2.0523
Epoch 15/50, Train Loss: 0.0107, Test Loss: 0.4035
Epoch 20/50, Train Loss: 0.0084, Test Loss: 0.0416
Early stopping triggered at epoch 24
Epoch 5/50, Train Loss: 0.0152, Test Loss: 0.2302
Epoch 10/50, Train Loss: 0.0127, Test Loss: 0.1595
Epoch 15/50, Train Loss: 0.0106, Test Loss: 0.5742
Early stopping triggered at epoch 20


In [5]:
def evaluate_model(model, test_loader, scaler_y):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    model.eval()

    y_true, y_pred = [], []

    with torch.no_grad():
        for X_batch, y_batch in test_loader:
            X_batch = X_batch.to(device)
            y_batch = y_batch.cpu().numpy()  
            
            y_pred_batch = model(X_batch).cpu().numpy()
            y_true.append(y_batch)
            y_pred.append(y_pred_batch)

    y_true = np.concatenate(y_true, axis=0)
    y_pred = np.concatenate(y_pred, axis=0)

    y_true = scaler_y.inverse_transform(y_true.reshape(-1, 1))
    y_pred = scaler_y.inverse_transform(y_pred.reshape(-1, 1))

    r2 = r2_score(y_true, y_pred)
    mae = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))

    print(f"\nEvaluation Results - R²: {r2:.4f}, MAE: {mae:.4f}, RMSE: {rmse:.4f}")

    return y_true, y_pred

y_true, y_pred = evaluate_model(model, stl_test_loader, stl_scaler_y)
y_true_hp, y_pred_hp = evaluate_model(model, hp_test_loader, hp_scaler_y)


Evaluation Results - R²: 0.9490, MAE: 0.0813, RMSE: 1.3891

Evaluation Results - R²: 0.9517, MAE: 0.0816, RMSE: 1.1157


# RNN/LSTM/GRU
Best results after several tries

In [6]:
# Create sequences for RNN models
def create_sequences(df, feature_cols, target_col, seq_length=14):
    X, y = [], []
    for i in range(len(df) - seq_length):
        X.append(df[feature_cols].iloc[i:i+seq_length].values)  
        y.append(df[target_col].iloc[i+seq_length])  
    
    return np.array(X), np.array(y).reshape(-1, 1)

stl_X, stl_y = create_sequences(stl_df, feature_cols, target_col, seq_length=14)
X_train, X_test, y_train, y_test = train_test_split(stl_X, stl_y, test_size=0.2, random_state=42)
hp_X_train, hp_X_test, hp_y_train, hp_y_test, hp_scaler_y = prepare_data(hp_df)

In [7]:
X_train, X_test, y_train, y_test = map(
    lambda x: torch.tensor(x, dtype=torch.float32).to("cuda" if torch.cuda.is_available() else "cpu"),
    (X_train, X_test, y_train, y_test)
)

batch_size = 64
train_loader = DataLoader(TensorDataset(X_train, y_train), batch_size=batch_size, shuffle=True)
test_loader = DataLoader(TensorDataset(X_test, y_test), batch_size=batch_size, shuffle=False)

In [8]:
class RecurrentModel(nn.Module):
    def __init__(self, input_dim, hidden_dim=128, num_layers=2, model_type="RNN", dropout_rate=0.3):
        super(RecurrentModel, self).__init__()
        self.model_type = model_type
        self.hidden_dim = hidden_dim
        
        if model_type == "RNN":
            self.rnn = nn.RNN(input_dim, hidden_dim, num_layers=num_layers, batch_first=True, dropout=dropout_rate)
        elif model_type == "LSTM":
            self.rnn = nn.LSTM(input_dim, hidden_dim, num_layers=num_layers, batch_first=True, dropout=dropout_rate)
        elif model_type == "GRU":
            self.rnn = nn.GRU(input_dim, hidden_dim, num_layers=num_layers, batch_first=True, dropout=dropout_rate)

        self.layer_norm = nn.LayerNorm(hidden_dim)

        self.fc = nn.Sequential(
            nn.Linear(hidden_dim, 64),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(64, 1)
        )

    def forward(self, x):
        out, _ = self.rnn(x)

        if out.dim() == 3: 
            out = out[:, -1, :]  
        elif out.dim() == 2: 
            out = out.unsqueeze(1) 

        out = self.layer_norm(out)
        return self.fc(out)

In [9]:
def train_model(model, train_loader, test_loader, num_epochs=50, patience=7):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    
    criterion = nn.MSELoss()
    optimizer = optim.AdamW(model.parameters(), lr=0.001, weight_decay=1e-4)
    
    train_losses, test_losses = [], []
    print(f" Starting training for {num_epochs} epochs...")
    
    for epoch in range(num_epochs):
        model.train()
        epoch_loss = 0

        for X_batch, y_batch in train_loader:
            seq_length = 1  
            X_batch = X_batch.view(X_batch.shape[0], seq_length, -1)
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)

            optimizer.zero_grad()
            y_pred = model(X_batch)  
            loss = criterion(y_pred, y_batch)
            loss.backward()
            optimizer.step()

            epoch_loss += loss.item()

        train_losses.append(epoch_loss / len(train_loader))
        model.eval()
        test_loss = 0
        with torch.no_grad():
            for X_batch, y_batch in test_loader:
                X_batch = X_batch.view(X_batch.shape[0], seq_length, -1).to(device)
                y_batch = y_batch.to(device)
                y_pred = model(X_batch)
                test_loss += criterion(y_pred, y_batch).item()
        
        test_loss /= len(test_loader)
        test_losses.append(test_loss)

    
        if (epoch + 1) % 5 == 0:
            print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {train_losses[-1]:.4f}, Test Loss: {test_losses[-1]:.4f}")

        # Early stopping check
        if len(test_losses) > patience and test_losses[-1] > min(test_losses[-patience:]):
            print(f" Early stopping triggered at epoch {epoch+1}")
            break

    return train_losses, test_losses

train_losses_stl_after, test_losses_stl_after = train_model(model, stl_train_loader, stl_test_loader, num_epochs=50)
train_losses_hp_after, test_losses_hp_after = train_model(model, hp_train_loader, hp_test_loader, num_epochs=50)

 Starting training for 50 epochs...
Epoch 5/50, Train Loss: 0.0045, Test Loss: 0.5502
 Early stopping triggered at epoch 8
 Starting training for 50 epochs...
Epoch 5/50, Train Loss: 0.0078, Test Loss: 0.0025
 Early stopping triggered at epoch 8


In [48]:
def evaluate_model(model, test_loader, scaler_y):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    model.eval()

    y_true, y_pred = [], []

    with torch.no_grad():
        for X_batch, y_batch in test_loader:
            X_batch = X_batch.to(device)
            y_pred_batch = model(X_batch).cpu().numpy()
            
            if y_pred_batch.ndim == 3:
                y_pred_batch = y_pred_batch.squeeze(1)
                
            y_true.append(y_batch.cpu().numpy())
            y_pred.append(y_pred_batch)

    y_true = np.concatenate(y_true, axis=0)
    y_pred = np.concatenate(y_pred, axis=0)

    y_true = scaler_y.inverse_transform(y_true.reshape(-1, 1))
    y_pred = scaler_y.inverse_transform(y_pred.reshape(-1, 1))

    r2 = r2_score(y_true, y_pred)
    mae = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))

    print(f"Evaluation Results - R²: {r2:.4f}, MAE: {mae:.4f}, RMSE: {rmse:.4f}")
    return r2, mae, rmse
    
r2_stl, mae_stl, rmse_stl = evaluate_model(model, stl_test_loader, stl_scaler_y)
r2_hp, mae_hp, rmse_hp = evaluate_model(model, hp_test_loader, hp_scaler_y)

Evaluation Results - R²: 0.9983, MAE: 0.0533, RMSE: 0.2533
Evaluation Results - R²: 0.9975, MAE: 0.0622, RMSE: 0.2516


In [49]:
def evaluate_model_predictions(model, test_loader, scaler_y):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    model.eval()

    y_true, y_pred = [], []

    with torch.no_grad():
        for X_batch, y_batch in test_loader:
            X_batch = X_batch.to(device)
            y_pred_batch = model(X_batch).cpu().numpy()
            
            if y_pred_batch.ndim == 3:
                y_pred_batch = y_pred_batch.squeeze(1)
                
            y_true.append(y_batch.cpu().numpy())
            y_pred.append(y_pred_batch)

    y_true = np.concatenate(y_true, axis=0)
    y_pred = np.concatenate(y_pred, axis=0)

    y_true = scaler_y.inverse_transform(y_true.reshape(-1, 1))
    y_pred = scaler_y.inverse_transform(y_pred.reshape(-1, 1))

    return y_true, y_pred 


In [28]:
hidden_dim = 128  

models_stl = {}
models_hp = {}

for model_type in ["RNN", "LSTM", "GRU"]:
    print(f"\n Training {model_type} on STL Data...")
    model_stl = RecurrentModel(len(feature_cols), hidden_dim, model_type=model_type)
    train_losses_stl_after, test_losses_stl_after = train_model(model_stl, stl_train_loader, stl_test_loader, num_epochs=50)
    models_stl[model_type] = model_stl 
    
    print(f"\n Training {model_type} on HP Data...")
    model_hp = RecurrentModel(len(feature_cols), hidden_dim, model_type=model_type)
    train_losses_hp_after, test_losses_hp_after = train_model(model_hp, hp_train_loader, hp_test_loader, num_epochs=50)
    models_hp[model_type] = model_hp 



 Training RNN on STL Data...
 Starting training for 50 epochs...
Epoch 5/50, Train Loss: 0.0044, Test Loss: 4.5093
 Early stopping triggered at epoch 8

 Training RNN on HP Data...
 Starting training for 50 epochs...
Epoch 5/50, Train Loss: 0.0096, Test Loss: 4.1685
 Early stopping triggered at epoch 9

 Training LSTM on STL Data...
 Starting training for 50 epochs...
Epoch 5/50, Train Loss: 0.0031, Test Loss: 4.3595
 Early stopping triggered at epoch 9

 Training LSTM on HP Data...
 Starting training for 50 epochs...
Epoch 5/50, Train Loss: 0.0078, Test Loss: 4.2440
 Early stopping triggered at epoch 9

 Training GRU on STL Data...
 Starting training for 50 epochs...
Epoch 5/50, Train Loss: 0.0039, Test Loss: 4.6015
 Early stopping triggered at epoch 9

 Training GRU on HP Data...
 Starting training for 50 epochs...
Epoch 5/50, Train Loss: 0.0084, Test Loss: 4.1525
 Early stopping triggered at epoch 8


In [29]:
# STL Evaluation
r2_stl_rnn, mae_stl_rnn, rmse_stl_rnn = evaluate_model(models_stl["RNN"], stl_test_loader, stl_scaler_y)
r2_stl_lstm, mae_stl_lstm, rmse_stl_lstm = evaluate_model(models_stl["LSTM"], stl_test_loader, stl_scaler_y)
r2_stl_gru, mae_stl_gru, rmse_stl_gru = evaluate_model(models_stl["GRU"], stl_test_loader, stl_scaler_y)

# HP Evaluation
r2_hp_rnn, mae_hp_rnn, rmse_hp_rnn = evaluate_model(models_hp["RNN"], hp_test_loader, hp_scaler_y)
r2_hp_lstm, mae_hp_lstm, rmse_hp_lstm = evaluate_model(models_hp["LSTM"], hp_test_loader, hp_scaler_y)
r2_hp_gru, mae_hp_gru, rmse_hp_gru = evaluate_model(models_hp["GRU"], hp_test_loader, hp_scaler_y)


Evaluation Results - R²: 0.0885, MAE: 0.1993, RMSE: 5.8724
Evaluation Results - R²: 0.0931, MAE: 0.2195, RMSE: 5.8577
Evaluation Results - R²: 0.0728, MAE: 0.2212, RMSE: 5.9228
Evaluation Results - R²: 0.1141, MAE: 0.2083, RMSE: 4.7777
Evaluation Results - R²: 0.1136, MAE: 0.2924, RMSE: 4.7789
Evaluation Results - R²: 0.0939, MAE: 0.2356, RMSE: 4.8317


In [73]:
import os
import matplotlib.pyplot as plt

if not os.path.exists("DL"):
    os.makedirs("DL")

def save_plot(fig, filename):
    fig.savefig(f"DL/{filename}.png", bbox_inches="tight")
    plt.close(fig)

In [74]:
def save_loss_curves(train_losses, test_losses, title, filename):
    fig = plt.figure(figsize=(8, 5))
    plt.plot(range(1, len(train_losses) + 1), train_losses, label="Train Loss", marker="o")
    plt.plot(range(1, len(test_losses) + 1), test_losses, label="Test Loss", marker="s")
    plt.xlabel("Epochs")
    plt.ylabel("Loss")
    plt.title(title)
    plt.legend()
    plt.grid(True)
    save_plot(fig, filename)

In [75]:
save_loss_curves(train_losses_stl, test_losses_stl, "STL Before RNN/LSTM/GRU", "train_loss_stl_before")
save_loss_curves(train_losses_hp, test_losses_hp, "HP Before RNN/LSTM/GRU", "train_loss_hp_before")

save_loss_curves(train_losses_stl_after, test_losses_stl_after, "STL RNN Loss", "train_loss_stl_rnn")
save_loss_curves(train_losses_stl_after, test_losses_stl_after, "STL LSTM Loss", "train_loss_stl_lstm")
save_loss_curves(train_losses_stl_after, test_losses_stl_after, "STL GRU Loss", "train_loss_stl_gru")

save_loss_curves(train_losses_hp_after, test_losses_hp_after, "HP RNN Loss", "train_loss_hp_rnn")
save_loss_curves(train_losses_hp_after, test_losses_hp_after, "HP LSTM Loss", "train_loss_hp_lstm")
save_loss_curves(train_losses_hp_after, test_losses_hp_after, "HP GRU Loss", "train_loss_hp_gru")


In [81]:
def save_all_predictions(y_true, y_pred_rnn, y_pred_lstm, y_pred_gru, title, filename):
    fig = plt.figure(figsize=(10, 6))
    plt.plot(y_true, label="Actual", linestyle="dashed", color="black")
    plt.plot(y_pred_rnn, label="RNN Prediction", alpha=0.7)
    plt.plot(y_pred_lstm, label="LSTM Prediction", alpha=0.7)
    plt.plot(y_pred_gru, label="GRU Prediction", alpha=0.7)
    
    plt.xlabel("Samples")
    plt.ylabel("Predicted Values")
    plt.title(title)
    plt.legend()
    plt.grid(True)

    save_plot(fig, filename)


In [82]:
save_all_predictions(y_true_stl, y_pred_rnn_stl, y_pred_lstm_stl, y_pred_gru_stl, 
                     "STL - Model Predictions Comparison", "stl_model_predictions_comparison")

save_all_predictions(y_true_hp, y_pred_rnn_hp, y_pred_lstm_hp, y_pred_gru_hp, 
                     "HP - Model Predictions Comparison", "hp_model_predictions_comparison")

In [76]:
def save_predictions_plot(y_true, y_pred, title, filename):
    fig = plt.figure(figsize=(8, 5))
    plt.scatter(y_true, y_pred, alpha=0.6)
    plt.plot([min(y_true), max(y_true)], [min(y_true), max(y_true)], linestyle='--', color='red')
    plt.xlabel("Actual Values")
    plt.ylabel("Predicted Values")
    plt.title(title)
    plt.grid(True)
    save_plot(fig, filename)

In [77]:
save_predictions_plot(y_true_stl, y_pred_rnn_stl, "STL RNN Predictions", "stl_rnn_predictions")
save_predictions_plot(y_true_stl, y_pred_lstm_stl, "STL LSTM Predictions", "stl_lstm_predictions")
save_predictions_plot(y_true_stl, y_pred_gru_stl, "STL GRU Predictions", "stl_gru_predictions")

save_predictions_plot(y_true_hp, y_pred_rnn_hp, "HP RNN Predictions", "hp_rnn_predictions")
save_predictions_plot(y_true_hp, y_pred_lstm_hp, "HP LSTM Predictions", "hp_lstm_predictions")
save_predictions_plot(y_true_hp, y_pred_gru_hp, "HP GRU Predictions", "hp_gru_predictions")

In [83]:
def save_model_performance_comparison(models, r2_stl, mae_stl, rmse_stl, r2_hp, mae_hp, rmse_hp, filename):
    x = np.arange(len(models))  # Model positions on x-axis
    width = 0.2  # Bar width

    fig, ax = plt.subplots(figsize=(10, 6))

    # STL Metrics
    ax.bar(x - 2*width, r2_stl, width, label="STL R²", color="blue")
    ax.bar(x - width, mae_stl, width, label="STL MAE", color="green")
    ax.bar(x, rmse_stl, width, label="STL RMSE", color="purple")

    # HP Metrics
    ax.bar(x + width, r2_hp, width, label="HP R²", color="orange")
    ax.bar(x + 2*width, mae_hp, width, label="HP MAE", color="red")
    ax.bar(x + 3*width, rmse_hp, width, label="HP RMSE", color="brown")

    ax.set_xlabel("Model Type")
    ax.set_ylabel("Score")
    ax.set_title("Deep Learning Model Performance Comparison (STL vs HP)")
    ax.set_xticks(x)
    ax.set_xticklabels(models, rotation=0)
    ax.legend()

    plt.grid(axis="y", linestyle="--", alpha=0.7)

    plt.savefig(f"DL/{filename}.png", bbox_inches="tight")
    plt.close()

save_model_performance_comparison(["RNN", "LSTM", "GRU"], 
                                  [r2_stl_rnn, r2_stl_lstm, r2_stl_gru],  # STL R²
                                  [mae_stl_rnn, mae_stl_lstm, mae_stl_gru],  # STL MAE
                                  [rmse_stl_rnn, rmse_stl_lstm, rmse_stl_gru],  # STL RMSE
                                  [r2_hp_rnn, r2_hp_lstm, r2_hp_gru],  # HP R²
                                  [mae_hp_rnn, mae_hp_lstm, mae_hp_gru],  # HP MAE
                                  [rmse_hp_rnn, rmse_hp_lstm, rmse_hp_gru],  # HP RMSE
                                  "dl_model_performance_comparison")
